In [1]:
from ipywidgets import interact, interactive
from IPython.display import display
import ipywidgets as widgets
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
%matplotlib inline

In [2]:
dataPath = './BRATS2015_Training'

## 数据

### 先读个图出来看看

#### 选一个文件

In [3]:
def generatePathDictionary(startPath):
    if os.path.isdir(startPath):
        return {
            currentPath : generatePathDictionary(startPath  + '/' + currentPath) \
            for currentPath in os.listdir(startPath)
        }
    elif os.path.isfile(startPath):
#         print startPath
        return startPath
    else:
        print startPath
        raise ValueError

In [4]:
dic = generatePathDictionary(dataPath)

firstList = []
secondList = []
thirdList = []
lastList = []

def firstFunction(cfirst):
    firstList.append(cfirst)
    secondRange.options = dic[cfirst].keys()
    
def secondFunction(csecond):
    secondList.append(csecond)
    thirdRange.options = dic[firstList[-1]][csecond].keys()

def thirdFunction(cthird):
    thirdList.append(cthird)
    fouthRange.options = dic[firstList[-1]][secondList[-1]][cthird].keys()
    

    
def lastFunction(clast):
    lastList.append(
        dic[firstList[-1]][secondList[-1]][thirdList[-1]][clast])
    print lastList[-1]
    
firstRange = widgets.Select(options = dic.keys())
first = firstRange.value

secondRange = widgets.Select(options = dic[first].keys())
second = secondRange.value

thirdRange = widgets.Select(options = dic[first][second].keys())
third = thirdRange.value

fouthRange = widgets.Select(options = dic[first][second][third].keys())
fouth = fouthRange.value

display(interactive(firstFunction, cfirst = firstRange))
display(interactive(secondFunction, csecond = secondRange))
display(interactive(thirdFunction, cthird = thirdRange))
display(interactive(lastFunction, clast = fouthRange))

# interact(firstFunction, cfirst = firstRange)
# interact(secondFunction, csecond = secondRange)
# interact(thirdFunction, cthird = thirdRange)
# interact(lastFunction, clast = fouthRange)

./BRATS2015_Training


ValueError: 

**目前代码还不完善，需要手动将上面输出的路径拷贝到变量中**

In [5]:
chooseFilePath = '''./BRATS2015_Training/LGG/brats_tcia_pat152_0001/VSD.Brain.XX.O.MR_T1c.40837/VSD.Brain.XX.O.MR_T1c.40837.mha'''

#### 绘制

In [2]:
import ipyvolume.pylab as p3
import ipyvolume
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [7]:
def show3D(filePath):
    image = sitk.ReadImage(filePath)
    img = sitk.GetArrayFromImage(image)
    
    display(ipyvolume.volume.quickvolshow(img))
    

In [8]:
def showSlice(filePath):
    image = sitk.ReadImage(filePath)
    img = sitk.GetArrayFromImage(image)
    zSlice = img[1, :, :]
    pl = figure(
        plot_width=400, 
        plot_height=400, 
        x_range=(0, 10), 
        y_range=(0, 10))
    slicer = pl.image(
        image = [zSlice], 
        x=[0], 
        y=[0], 
        dw=[10], 
        dh=[10])
    
    def update(z):
        zSlicer = img[z, :, :]
        slicer.data_source.data['image'] = [zSlicer]
        push_notebook()
        
    show(pl, notebook_handle = True)
    play = widgets.Play(
        value=50,
        min=0,
        max=154,
        step=1,
        description="Press play",
        disabled=False)
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    interact(update, z = play)
    display(slider)

##### 切片图

In [17]:
def showSlice(img):
    zSlice = img[1, :, :]
    pl = figure(
        plot_width=400, 
        plot_height=400, 
        x_range=(0, 10), 
        y_range=(0, 10))
    slicer = pl.image(
        image = [zSlice], 
        x=[0], 
        y=[0], 
        dw=[10], 
        dh=[10])
    
    def update(z):
        zSlicer = img[z, :, :]
        slicer.data_source.data['image'] = [zSlicer]
        push_notebook()
        
    show(pl, notebook_handle = True)
    play = widgets.Play(
        value=50,
        min=0,
        max=154,
        step=1,
        description="Press play",
        disabled=False)
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    interact(update, z = play)
    display(slider)

In [3]:
filePath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/Val17-07-13_00:42:08/Brats17_CBICA_ABM_1/segmentResult.npy'

In [8]:
maskPath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/17-07-12_21:07:22/Brats17_2013_22_1/resultMask.npy'

In [7]:
labelPath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/Val17-07-13_00:42:08/Brats17_CBICA_ABM_1/gTArray.npy'

In [12]:
import nibabel as nib

In [13]:
labelArray = nib.load(labelPath)

In [14]:
labelArray = labelArray.get_data()

In [8]:
labelArray = np.load(labelPath)

In [9]:
maskArray = np.load(maskPath)

In [4]:
imageArray = np.load(filePath)

In [5]:
showSlice(imageArray)

NameError: name 'showSlice' is not defined

In [13]:
imageArray[125][125]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [14]:
labelArray[125][125]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [12]:
imageArray[120][120] != labelArray[120][120]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False,  True,  True,
        True,  True,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [24]:
import utils.metrics as ms

In [25]:
ms.voxleWiseMetrics(imageArray, labelArray, [1,2,3])

(0.29808998149941834, 0.91966902076525769, 0.9077006177431205)

In [6]:
display(ipyvolume.volume.quickvolshow(imageArray))

/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [9]:
display(ipyvolume.volume.quickvolshow(labelArray))

In [10]:
display(ipyvolume.volume.quickvolshow(maskArray))

In [9]:
showSlice(chooseFilePath)

RuntimeError: Exception thrown in SimpleITK ReadImage: /opt/miniconda2/conda-bld/simpleitk_1491578024069/work/Code/IO/src/sitkImageReaderBase.cxx:74:
sitk::ERROR: The file "./BRATS2015_Training/LGG/brats_tcia_pat152_0001/VSD.Brain.XX.O.MR_T1c.40837/VSD.Brain.XX.O.MR_T1c.40837.mha" does not exist.

##### 3D交互式绘图

In [10]:
show3D(chooseFilePath)

/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)
